In [1]:
import os
from os import getcwd

import tensorflow as tf

In [2]:
try:
    os.mkdir('/tmp/Liver')
    os.mkdir('/tmp/Liver/training')
    os.mkdir('/tmp/Liver/training/f0')
    os.mkdir('/tmp/Liver/training/f1')
    os.mkdir('/tmp/Liver/training/f2')
    os.mkdir('/tmp/Liver/training/f3')
    os.mkdir('/tmp/Liver/training/f4')
    os.mkdir('/tmp/Liver/testing')
    os.mkdir('/tmp/Liver/testing/f0')
    os.mkdir('/tmp/Liver/testing/f1')
    os.mkdir('/tmp/Liver/testing/f2')
    os.mkdir('/tmp/Liver/testing/f3')
    os.mkdir('/tmp/Liver/testing/f4')
    
    #YOUR CODE GOES HERE
except OSError:
    pass

In [3]:
import random
from shutil import copyfile

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    dataset = []
    
    for unitData in os.listdir(SOURCE):
        data = SOURCE + unitData
        if(os.path.getsize(data) > 0):
            dataset.append(unitData)
        else:
            print('Skipped ' + unitData)
            print('Invalid file i.e zero size')
    
    train_set_length = int(len(dataset) * SPLIT_SIZE)
    test_set_length = int(len(dataset) - train_set_length)
    shuffled_set = random.sample(dataset, len(dataset))
    train_set = dataset[0:train_set_length]
    test_set = dataset[-test_set_length:]
       
    for unitData in train_set:
        temp_train_set = SOURCE + unitData
        final_train_set = TRAINING + unitData
        copyfile(temp_train_set, final_train_set)
    
    for unitData in test_set:
        temp_test_set = SOURCE + unitData
        final_test_set = TESTING + unitData
        copyfile(temp_test_set, final_test_set)

In [4]:
F0_SOURCE_DIR = "../../../Data/Liver/0/"
TRAINING_F0_DIR = "/tmp/Liver/training/f0/"
TESTING_F0_DIR = "/tmp/Liver/testing/f0/"

F1_SOURCE_DIR = "../../../Data/Liver/1/"
TRAINING_F1_DIR = "/tmp/Liver/training/f1/"
TESTING_F1_DIR = "/tmp/Liver/testing/f1/"

F2_SOURCE_DIR = "../../../Data/Liver/2/"
TRAINING_F2_DIR = "/tmp/Liver/training/f2/"
TESTING_F2_DIR = "/tmp/Liver/testing/f2/"

F3_SOURCE_DIR = "../../../Data/Liver/3/"
TRAINING_F3_DIR = "/tmp/Liver/training/f3/"
TESTING_F3_DIR = "/tmp/Liver/testing/f3/"

F4_SOURCE_DIR = "../../../Data/Liver/4/"
TRAINING_F4_DIR = "/tmp/Liver/training/f4/"
TESTING_F4_DIR = "/tmp/Liver/testing/f4/"

split_size = .8

split_data(F0_SOURCE_DIR, TRAINING_F0_DIR, TESTING_F0_DIR, split_size)
split_data(F1_SOURCE_DIR, TRAINING_F1_DIR, TESTING_F1_DIR, split_size)
split_data(F2_SOURCE_DIR, TRAINING_F2_DIR, TESTING_F2_DIR, split_size)
split_data(F3_SOURCE_DIR, TRAINING_F3_DIR, TESTING_F3_DIR, split_size)
split_data(F4_SOURCE_DIR, TRAINING_F4_DIR, TESTING_F4_DIR, split_size)

In [5]:
print("Training Fibrosis 0:", len(os.listdir(TRAINING_F0_DIR)))
print("Testing Fibrosis 0:", len(os.listdir(TESTING_F0_DIR)))
print("-"*20)
print("Training Fibrosis 1:", len(os.listdir(TRAINING_F1_DIR)))
print("Testing Fibrosis 1:", len(os.listdir(TESTING_F1_DIR)))
print("-"*20)
print("Training Fibrosis 2:", len(os.listdir(TRAINING_F2_DIR)))
print("Testing Fibrosis 2:", len(os.listdir(TESTING_F2_DIR)))
print("-"*20)
print("Training Fibrosis 3:", len(os.listdir(TRAINING_F3_DIR)))
print("Testing Fibrosis 3:", len(os.listdir(TESTING_F3_DIR)))
print("-"*20)
print("Training Fibrosis 4:", len(os.listdir(TRAINING_F4_DIR)))
print("Testing Fibrosis 4:", len(os.listdir(TESTING_F4_DIR)))
print("-"*20)

Training Fibrosis 0: 889
Testing Fibrosis 0: 223
--------------------
Training Fibrosis 1: 111
Testing Fibrosis 1: 28
--------------------
Training Fibrosis 2: 108
Testing Fibrosis 2: 27
--------------------
Training Fibrosis 3: 361
Testing Fibrosis 3: 91
--------------------
Training Fibrosis 4: 728
Testing Fibrosis 4: 183
--------------------


In [7]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.99):
            print("\nReached 99.0% accuracy so cancelling training!")
            self.model.stop_training = True

In [8]:
from tensorflow.keras.optimizers import RMSprop

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(600,800,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
    
])

In [10]:
"""
model.compile(optimizer=RMSprop(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['acc'])
"""

"\nmodel.compile(optimizer=RMSprop(lr=0.001),\n              loss='categorical_crossentropy',\n              metrics=['acc'])\n"

In [11]:

optimizer = RMSprop(lr=1e-8, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=["acc"])         


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 598, 798, 16)      160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 299, 399, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 297, 397, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 148, 198, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 146, 196, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 73, 98, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 71, 96, 64)        3

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

TRAINING_DIR = '/tmp/Liver/training'
train_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                   batch_size=64,
                                                   class_mode='categorical',
                                                   target_size=(600,800),
                                                   color_mode='grayscale') # target_size = (height, width)

TESTING_DIR = '/tmp/Liver/testing'
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

test_generator = train_datagen.flow_from_directory(TESTING_DIR,
                                                   batch_size=16,
                                                   class_mode='categorical',
                                                   target_size=(600,800), 
                                                   color_mode='grayscale')

Found 2197 images belonging to 5 classes.
Found 552 images belonging to 5 classes.


In [ ]:
callbacks = myCallback()

history = model.fit_generator(train_generator,
                              epochs=50,
                              validation_data=test_generator,
                              callbacks=[callbacks])

Epoch 1/50
35/35 [==============================] - 11s 318ms/step - loss: 1.6066 - acc: 0.3773 - val_loss: 1.6055 - val_acc: 0.3822
Epoch 2/50
35/35 [==============================] - 10s 295ms/step - loss: 1.6049 - acc: 0.3987 - val_loss: 1.6039 - val_acc: 0.3967
Epoch 3/50
35/35 [==============================] - 10s 297ms/step - loss: 1.6032 - acc: 0.4033 - val_loss: 1.6023 - val_acc: 0.3986
Epoch 4/50
35/35 [==============================] - 10s 297ms/step - loss: 1.6015 - acc: 0.4046 - val_loss: 1.6007 - val_acc: 0.4022
Epoch 5/50
35/35 [==============================] - 10s 298ms/step - loss: 1.6000 - acc: 0.4046 - val_loss: 1.5992 - val_acc: 0.4022
Epoch 6/50
35/35 [==============================] - 10s 297ms/step - loss: 1.5985 - acc: 0.4046 - val_loss: 1.5977 - val_acc: 0.4022
Epoch 7/50
35/35 [==============================] - 10s 299ms/step - loss: 1.5969 - acc: 0.4046 - val_loss: 1.5962 - val_acc: 0.4040
Epoch 8/50
35/35 [==============================] - 10s 297ms/step - 

In [ ]:
# check the proper learning rate(lr)

import matplotlib.pyplot as plt

plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-4, 0, 60])

In [19]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
    
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

NameError: name 'history' is not defined

In [ ]:
import os, signal

os.kill(os.getpid(),signal.SIGKILL)